In [ ]:
import numpy as np
import h5py
import math
import pandas as pd
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import os, sys
import random
import scipy
from PIL import Image
from PIL import ImageDraw
import time
import io

In [ ]:
# set locations for working files
if len(sys.argv) != 3:
    print("Usage: python3 process-sim.py <automation_dir> <attpcroot_dir>")
    print('Assuming testing directories')
    automation_dir = '/mnt/analysis/e17023/Adam/GADGET/.sims/0/'
    attpcroot_dir = automation_dir + 'ATTPCROOTv2/'
else:
    # Automation directory
    automation_dir = sys.argv[1]
    
    # ATTPCROOTv2 directory
    attpcroot_dir = sys.argv[2]

In [ ]:
def indicator_file(file_type, indicator_directory=automation_dir):
    # remove old indicator file(s)
    for file in os.listdir(indicator_directory):
        if file.endswith('.tmp'):
            os.remove(indicator_directory + file)
    
    with open(indicator_directory + file_type + '.tmp', 'w') as f:
        f.write('1')
    if file_type == 'STOP':
        print('STOPPING')
        sys.exit()
    return None

In [ ]:
parameters = pd.read_csv(automation_dir + 'param.csv')

In [ ]:
# check for and complete any active simulations

# 0 = inactive
# 1 = active
# 2 = complete
indicator_file('PROCESSING H5')

active_sims = parameters[parameters['Status'] == 1]

if len(active_sims) > 1:
    print('More than one simulation marked as active')
    indicator_file('STOP')

# Search for output.h5 and rename
Complete = False
try:
    os.rename(automation_dir + 'out/output.h5', automation_dir + 'out/hdf5/' + active_sims.loc[active_sims.index[0],'Sim']+'.h5')
    Complete = True
except FileNotFoundError:
    pass

# Set Status in parameters
if Complete:
    parameters.loc[active_sims.index[0], 'Status'] = 2
    if 'Time' in parameters.columns:
        parameters.loc[active_sims.index[0], 'Time'] = time.time() - parameters.loc[active_sims.index[0], 'Time']
else:
    print('No output.h5 found')
    indicator_file('STOP')

# Update parameters.csv to reflect complete h5 file
parameters.to_csv(automation_dir + 'param.csv', index=False)

In [ ]:
def save_cutImages(cut_indices, chunk_num):

			def make_box(mm_grid):
				"""
				Draws the box for the energy bar
				"""
				box_row = np.array([4, 4])
				to_box_row = np.array([145, 146])
				for_box_col = np.array([7, 17])

				box_col = np.array([7, 7])
				to_box_col = np.array([17, 17])
				for_box_row = np.array([4, 145])

				# Draw vertical lines of energy bar box
				for i in range(len(box_row)):
					mm_grid[box_row[i]:to_box_row[i], for_box_col[i], :] = 0
					mm_grid[for_box_row[i], box_col[i]:to_box_col[i], :] = 0

				return mm_grid


			def make_grid():
				"""
				"Create Training Data.ipynb"eate grid matrix of MM outline and energy bar, see spreadsheet below
				https://docs.google.com/spreadsheets/d/1_bbg6svfEph_g_Z002rmzTLu8yjQzuj_p50wqs7mMrI/edit?usp=sharing
				"""
				row = np.array([63, 47, 39, 31, 27, 23, 19, 15, 15, 11, 11, 7, 7, 7, 7, 3, 3, 3, 3, 3, 3, 3, 7, 7, 
					    7, 7, 11, 11, 15, 15, 19, 23, 27, 31, 39, 47, 63]) 

				to_row = np.array([87, 103, 111, 119, 123, 127, 131, 135, 135, 139, 139, 143, 143, 143, 143, 147, 
					       147, 147, 147, 147, 147, 148, 143, 143, 143, 144, 139, 140, 135, 136, 132, 128,
					       124, 120, 112, 104, 88]) 

				col = np.array([100, 84, 76, 68, 64, 60, 56, 52, 52, 48, 48, 44, 44, 44, 44, 40, 40, 40, 40, 40, 
					    40, 40, 44, 44, 44, 44, 48, 48, 52, 52, 56, 60, 64, 68, 76, 84, 100])

				to_col = np.array([124, 140, 148, 156, 160, 164, 168, 172, 172, 176, 176, 180, 180, 180, 180, 184, 
					       184, 184, 184, 184, 184, 184, 180, 180, 180, 180, 176, 176, 172, 172, 168, 164, 
					       160, 156, 148, 140, 124]) 

				all_row = np.array([i for i in range(3, 148, 4)])
				all_col = np.array([i for i in range(40, 185, 4)]) 

				full_image_size_width = 224
				full_image_size_length = 151
				mm_grid = np.zeros((full_image_size_length, full_image_size_width, 3))    
				mm_grid.fill(255)                                                     

				for i in range(len(row)):
					# draw grid columns, 0 = black
					mm_grid[row[i]:to_row[i], all_col[i], :] = 0
			
				for i in range(len(col)):
					# draw grid rows
					mm_grid[all_row[i], col[i]:to_col[i], :] = 0

				# Ensure that matrix is of integers
				mm_grid = mm_grid.astype(int) 

				# Draw engergy bar box
				mm_grid = make_box(mm_grid)

				return mm_grid

			# Precompute the grid once and reuse it
			global_grid = make_grid()


			def blue_range(pad_plane, rows):

				start_row = 140
				low_color = 0
				high_color = 35

				for i in range(rows):
					pad_plane[start_row:start_row+5, 8:17, 0] = low_color
					pad_plane[start_row:start_row+5, 8:17, 1] = high_color
					start_row = start_row - 5 
					low_color = low_color + 35
					high_color = high_color + 35

				return pad_plane


			def yellow_range(pad_plane, rows):

				start_row = 105
				color = 220

				for i in range(rows):
					pad_plane[start_row:start_row+5, 8:17, 2] = color
					start_row = start_row - 5 
					color = color - 15

				return pad_plane


			def orange_range(pad_plane, rows):

				start_row = 70
				color = 210
				for i in range(rows):
					pad_plane[start_row:start_row+5, 8:17, 1] = color - 15
					pad_plane[start_row:start_row+5, 8:17, 2] = color
					start_row = start_row - 5 
					color = color - 15

				return pad_plane


			def red_range(pad_plane, rows):

				start_row = 35
				color = 250

				for i in range(rows):
					pad_plane[start_row:start_row+5, 8:17, 0] = color
					pad_plane[start_row:start_row+5, 8:17, 1] = 50
					pad_plane[start_row:start_row+5, 8:17, 2] = 50
					start_row = start_row - 5 
					color = color - 15

				return pad_plane


			def tot_energy_to_mev(tot_energy):
				tot_energy = tot_energy / 1000
				return tot_energy


			def fill_energy_bar(pad_plane, tot_energy):
				"""
				Fills the energy bar where the amount of pixels fired and the color corresponds to the energy of the track
				Max pixel_range should be 28 (7 rows for each color), so need to adjust accordingly.
				"""
				# Calculate the energy in MeV
				energy_mev = tot_energy_to_mev(tot_energy)

				# Calculate the proportion of the energy bar that should be filled
				proportion_filled = energy_mev / 3

				# Calculate how many rows should be filled
				total_rows = math.floor(proportion_filled * 28)

				# Fill the energy bar one row at a time
				if total_rows > 0:
					pad_plane = blue_range(pad_plane, rows=min(total_rows, 7))
				if total_rows > 7:
					pad_plane = yellow_range(pad_plane, rows=min(total_rows-7, 7))
				if total_rows > 14:
					pad_plane = orange_range(pad_plane, rows=min(total_rows-14, 7))
				if total_rows > 21:
					pad_plane = red_range(pad_plane, rows=min(total_rows-21, 7))

				return pad_plane


			def pos_odd_even(event_value):
				"""
				Makes correction to positive points if they are odd or even
				"""
				if event_value % 2 == 0:
					event_value = event_value + 1
					return event_value

				else:
					return event_value


			def neg_odd_even(event_value):
				"""
				Makes correction to negative points if they are odd or even
				"""
				if event_value % 2 == 0:
					event_value = event_value - 1
					return event_value

				else:
					return event_value


			def pt_shift(xset, yset):
				"""
				Shifts all points to the center of nearest pad for pad mapping
				"""
				for j in range(len(xset)):

					if xset[j] > 0:
						xset[j] = math.floor(xset[j])
						pos_adj_valx = pos_odd_even(xset[j])
						xset[j] = pos_adj_valx

					elif xset[j] < 0:
						xset[j] = math.ceil(xset[j])
						neg_adj_valx = neg_odd_even(xset[j])
						xset[j] = neg_adj_valx

					if yset[j] > 0:
						yset[j] = math.floor(yset[j])
						pos_adj_valy = pos_odd_even(yset[j])
						yset[j] = pos_adj_valy

					elif yset[j] < 0:
						yset[j] = math.ceil(yset[j])
						neg_adj_valy = neg_odd_even(yset[j])
						yset[j] = neg_adj_valy

				return xset, yset


			def fill_padplane(xset, yset, eset, tot_energy, global_grid):
				"""
				Fills the 2D pad plane grid for image creation
				"""
				pad_plane = np.copy(global_grid)

				xset = np.array(xset)
				yset = np.array(yset)
				eset = np.array(eset)

				# pad plane mapping
				x = (35 + xset) * 2 + 42    # col value
				y = 145 - (35 + yset) * 2   # row value

				# create a dictionary to store (x,y) as keys and e as values
				d = {}
				for i in range(len(x)):
					key = (x[i], y[i])
					if key in d:
						d[key] += eset[i]
					else:
						d[key] = eset[i]

				# convert the dictionary back to arrays
				x = np.zeros(len(d))
				y = np.zeros(len(d))
				eset = np.zeros(len(d))
				for i, key in enumerate(d):
					x[i] = key[0]
					y[i] = key[1]
					eset[i] = d[key]

				# Find max E value and normalize
				energy = eset
				max_energy = np.max(energy)
				norm_energy = energy / max_energy


				# Fill in pad plane   
				for k in range(len(x)):
				
					if y[k] < 9:
						y[k] = y[k] + 4

					if x[k] < 50:
						x[k] = x[k] + 4

					if x[k] > 174:
						x[k] = x[k] - 4

					if y[k] > 53:
						y[k] = y[k] - 4

					if x[k] > 134:
						x[k] = x[k] - 4

					if y[k] > 93:
						y[k] = y[k] - 4

					if y[k] > 133:
						y[k] = y[k] - 4	

					if x[k] < 90:
						x[k] = x[k] + 4

					yoff = 4
					xoff = 0
					pad_plane[int(y[k])-1+yoff:int(y[k])+2+yoff, int(x[k])-1+xoff:int(x[k])+2+xoff, 0] = norm_energy[k] * 205

					pad_plane[int(y[k])-1+yoff:int(y[k])+2+yoff, int(x[k])-1+xoff:int(x[k])+2+xoff, 1] = norm_energy[k] * 240
				
				pad_plane = fill_energy_bar(pad_plane, tot_energy)

				return pad_plane

			def drop_zeros(array):
				last_nonzero = np.max(np.nonzero(array))
				return array[:last_nonzero+1]

			def plot_track(cut_indices):
				import pickle
				all_image_data = []  # List to store the results
				xHit_list = np.load(os.path.join(automation_dir, 'out/xHit_list.npy'), allow_pickle=True)
				yHit_list = np.load(os.path.join(automation_dir, 'out/yHit_list.npy'), allow_pickle=True)
				eHit_list = np.load(os.path.join(automation_dir, 'out/eHit_list.npy'), allow_pickle=True)
				trace_list = np.load(os.path.join(automation_dir, 'out/trace_list.npy'), allow_pickle=True)
				tot_energy = np.load(os.path.join(automation_dir, 'out/tot_energy.npy'), allow_pickle=True)
    
				for event_num in cut_indices:
					xHit = drop_zeros(xHit_list[event_num])
					yHit = drop_zeros(yHit_list[event_num])
					eHit = drop_zeros(eHit_list[event_num])
     
					

					trace = trace_list[event_num]
					energy = tot_energy[event_num]

					# Call pt_shift function to move all 2D pts to pad centers
					dset_0_copyx, dset_0_copyy = pt_shift(xHit, yHit)

					# Call fill_padplane to create 2D pad plane image
					pad_plane = fill_padplane(dset_0_copyx, dset_0_copyy, eHit, energy, global_grid)

					# Prepare the data necessary for plotting
					image_title = f' Image {cut_indices[event_num]} of Particle Track Event'
					image_filename = f"{sim_name}_image_{event_num}.png"

					all_image_data.append((pad_plane, trace, image_title, image_filename))  # Append the result to the list

				del xHit_list 
				del yHit_list 
				del eHit_list

				return all_image_data  # Return the list of all results after the loop


			result = plot_track(cut_indices)
			return result

In [ ]:
def clean_trace(data_trace): 
    trace = data_trace
    max_val = np.argmax(trace)
    low_bound = max_val - 75
    if low_bound < 0:
	    low_bound = 5
    upper_bound = max_val + 75
    if upper_bound > 511:
        upper_bound = 506
    trace = trace[low_bound:upper_bound]
    return trace

In [ ]:
# CREATE IMAGES FROM HDF5 FILE
if Complete:
    indicator_file('PROCESSING IMAGES')
    sim_name = active_sims.loc[active_sims.index[0],'Sim']
    h5file = h5py.File(automation_dir + "out/hdf5/" + sim_name + '.h5', 'r')
    
    xHit_list = []; yHit_list = []; eHit_list = []; trace_list = []; tot_energy = []; cut_indices = []
    for i in h5file.keys():
        xHit_list.append(h5file[i]['HitArray']['x'])
        yHit_list.append(h5file[i]['HitArray']['y'])
        eHit_list.append(h5file[i]['HitArray']['A'])
        
        trace_list.append(clean_trace(h5file[i]['Trace']))
        tot_energy.append(active_sims.loc[active_sims.index[0],'E0'] + active_sims.loc[active_sims.index[0],'E1'] * random.uniform(0.95, 1.05))
        
        if len(trace_list[-1]) > 2:
            cut_indices.append(int(i.split("[")[1].split("]")[0]))
    
    xHit_list = np.array(xHit_list, dtype=object)
    yHit_list = np.array(yHit_list, dtype=object)
    eHit_list = np.array(eHit_list, dtype=object)
    trace_list = np.array(trace_list, dtype=object)
    tot_energy = np.array(tot_energy)
    cut_indices = np.array(cut_indices)
    
    np.save(automation_dir + 'out/xHit_list.npy', xHit_list)
    np.save(automation_dir + 'out/yHit_list.npy', yHit_list)
    np.save(automation_dir + 'out/eHit_list.npy', eHit_list)
    np.save(automation_dir + 'out/trace_list.npy', trace_list)
    np.save(automation_dir + 'out/tot_energy.npy', tot_energy)
else:
    sys.exit()

In [ ]:
circle_mask = np.ones((224, 224, 4), dtype=bool)
for i in range(circle_mask.shape[0]):
    for j in range(circle_mask.shape[1]):
        if (i-75)**2+(j-112)**2 <= 66.709**2:
            circle_mask[i,j,:] = [False,False,False,False]

In [ ]:
def veto_check(image,circle_mask=circle_mask):
    # apply mask to image
    
    masked_image = image * circle_mask
    padplane = masked_image[0:151,37:188,0] # padplane region

    padplane[np.where(padplane==255)] = 0 # remove white
    
    veto = np.any(padplane != 0) # check for nonzero values
    return veto

In [ ]:
# Process images in chunks to avoiding overloading memory
chunk_size = 500
num_images = len(cut_indices)
#print("Total Number of Image:", num_images)
num_chunks = (num_images + chunk_size - 1) // chunk_size
#print("Total Number of Chunks:", num_chunks)
chunk_num = 1
for chunk_idx in range(num_chunks):
	#print(f"Processing Chunk {chunk_idx+1} of {num_chunks}")
	start_idx = chunk_idx * chunk_size
	end_idx = min((chunk_idx + 1) * chunk_size, num_images)
	chunk_indices = cut_indices[start_idx:end_idx]
	image_data = save_cutImages(chunk_indices, chunk_num)
	my_dpi = 96
	fig_size = (224/my_dpi, 73/my_dpi)  # Fig size to be used in the main thread
	fig, ax = plt.subplots(figsize=fig_size)
	ax.tick_params(top=False, bottom=False, left=False, right=False, labelleft=False, labelbottom=False)
	ax.spines['top'].set_visible(False)
	ax.spines['right'].set_visible(False)
	ax.spines['bottom'].set_visible(False)
	ax.spines['left'].set_visible(False)
	# Plot and save images in the main thread
	for pad_plane, trace, title, filename in image_data:
		# Plot trace
		ax.clear()
		trace = np.nan_to_num(trace.astype(float))
		x = np.linspace(0, len(trace)-1, len(trace))
		ax.fill_between(x, trace, color='b', alpha=1)
		
		# Assuming pad_plane is a NumPy array with shape (height, width, 3)
		alpha_channel = np.ones((pad_plane.shape[0], pad_plane.shape[1], 1), dtype=pad_plane.dtype) * 255
		pad_plane_rgba = np.concatenate((pad_plane, alpha_channel), axis=2)
		
		# Now pad_plane_rgba has an additional alpha channel and can be concatenated with trace_img
		# Ensure trace image is saved as PNG and read it
		buf = io.BytesIO()
		fig.savefig(buf, format='png', dpi=my_dpi)
		buf.seek(0)
		with Image.open(buf) as im:
			trace_img_png = np.array(im)
		buf.close()
		# Concatenate pad_plane_rgba and trace_img_png
		complete_image = np.append(pad_plane_rgba, trace_img_png, axis=0)
		# Convert image data to uint8 if it's not already
		if complete_image.dtype != np.uint8:
			if complete_image.max() > 1:
				complete_image = complete_image.astype(np.uint8)
			else:
				complete_image = (255 * complete_image).astype(np.uint8)
		
		# Emulate dead pixels from the detector
		pads = [[11,23],[24,12]]
		for pad in pads:
			complete_image[4*pad[0]+4:4*pad[0]+7,4*pad[1]+41:4*pad[1]+44,:] = [255,255,255,255]

		# check if image should be vetoed due to edge proximity
		if not veto_check(complete_image):
			# Save the final concatenated image as PNG
			plt.imsave(os.path.join(automation_dir+"out/images/", filename), complete_image)
			print(f"Saved {filename}")
		else:
			print(f"Vetoed {filename}")
		# Close the figure to free memory
		plt.close(fig)
	chunk_num += 1

In [ ]:
# set status of active sim to 3
parameters.loc[active_sims.index[0], 'Status'] = 3 # complete images
parameters.to_csv(automation_dir + 'param.csv', index=False)
# remove temporary files
os.remove(automation_dir + 'out/xHit_list.npy')
os.remove(automation_dir + 'out/yHit_list.npy')
os.remove(automation_dir + 'out/eHit_list.npy')
os.remove(automation_dir + 'out/trace_list.npy')
os.remove(automation_dir + 'out/tot_energy.npy')

In [ ]:
# attempt to make gif of images
indicator_file('PROCESSING GIF')
image_list = os.listdir(automation_dir + 'out/images/')
image_list = [i for i in image_list if i.split('_')[0] == sim_name]
if len(image_list) > 100: # limit number of images to 100
    image_list = image_list[:100]
if len(image_list) < 2:
    sys.exit() # not enough images to make gif
frames = []
for i in range(len(image_list)):
    new_frame = Image.open(automation_dir + "out/images/" + image_list[i])
    frames.append(new_frame)
frames[0].save(f'{automation_dir}out/gifs/{sim_name}.gif', format='GIF', append_images=frames[1:], save_all=True, duration=100, loop=0)

# set status of active sim to 4
parameters.loc[active_sims.index[0], 'Status'] = 4 # complete gif
parameters.to_csv(automation_dir + 'param.csv', index=False)